In [67]:
import pandas as pd
import numpy as np

# read html

In [68]:
dfs = pd.read_html('../in/lotteria.html')
dfs

[       0       1         2      3          4
 0  Serie  Numero  Località  Prov.     Premio
 1      T  173756  Somaglia     LO  5.000.000
 2      T  378442    Pesaro     PU  2.500.000
 3      G  330068   Palermo     PA  2.000.000
 4      G  173817    Torino     TO  1.500.000
 5      S  185025      Dolo     VE  1.000.000,
      0                  1                  2                                               3                  4   5
 0  NaN  SECONDA CATEGORIA  SECONDA CATEGORIA                               SECONDA CATEGORIA  SECONDA CATEGORIA NaN
 1  NaN              Serie          Biglietto                                       Venduto a             Premio NaN
 2  NaN                  S             030585  DISTRIBUTORE LOCALE MONTENERO DI BISACCIA (CB)            100.000 NaN
 3  NaN                  C             439458                                   AVELLINO (AV)            100.000 NaN
 4  NaN                  M             217442                                   MASSAFRA (TA

In [69]:
pd.options.display.width = 200
for idx, df in enumerate(dfs):
    print(f'------------------- {idx} -----------------------')
    print(df.columns)
    print(df.head(3))

------------------- 0 -----------------------
Index([0, 1, 2, 3, 4], dtype='int64')
       0       1         2      3          4
0  Serie  Numero  Località  Prov.     Premio
1      T  173756  Somaglia     LO  5.000.000
2      T  378442    Pesaro     PU  2.500.000
------------------- 1 -----------------------
Index([0, 1, 2, 3, 4, 5], dtype='int64')
    0                  1                  2                                               3                  4   5
0 NaN  SECONDA CATEGORIA  SECONDA CATEGORIA                               SECONDA CATEGORIA  SECONDA CATEGORIA NaN
1 NaN              Serie          Biglietto                                       Venduto a             Premio NaN
2 NaN                  S             030585  DISTRIBUTORE LOCALE MONTENERO DI BISACCIA (CB)            100.000 NaN
------------------- 2 -----------------------
Index([0, 1, 2, 3, 4, 5], dtype='int64')
    0                1                2                                  3                4   5
0 NaN 

# category 1

In [70]:
cat1 = dfs[0]
cat1

,0,1,2,3,4
0,Serie,Numero,Località,Prov.,Premio
1,T,173756,Somaglia,LO,5.000.000
2,T,378442,Pesaro,PU,2.500.000
3,G,330068,Palermo,PA,2.000.000
4,G,173817,Torino,TO,1.500.000
5,S,185025,Dolo,VE,1.000.000


## get column names

In [71]:
col_names = cat1.iloc[0]
col_names

0       Serie
1      Numero
2    Località
3       Prov.
4      Premio
Name: 0, dtype: object

## rename columns, drop header

In [72]:
cat1.columns = col_names
cat1

,Serie,Numero,Località,Prov.,Premio
0,Serie,Numero,Località,Prov.,Premio
1,T,173756,Somaglia,LO,5.000.000
2,T,378442,Pesaro,PU,2.500.000
3,G,330068,Palermo,PA,2.000.000
4,G,173817,Torino,TO,1.500.000
5,S,185025,Dolo,VE,1.000.000


In [73]:
cat1 = cat1.drop(index=0)
cat1

,Serie,Numero,Località,Prov.,Premio
1,T,173756,Somaglia,LO,5.000.000
2,T,378442,Pesaro,PU,2.500.000
3,G,330068,Palermo,PA,2.000.000
4,G,173817,Torino,TO,1.500.000
5,S,185025,Dolo,VE,1.000.000


In [74]:
if cat1.Premio.dtype == 'O':
    cat1.Premio = cat1.Premio.str.replace('.', '').astype(int)
cat1.dtypes

0
Serie       object
Numero      object
Località    object
Prov.       object
Premio       int64
dtype: object

In [75]:
cat1.head()

,Serie,Numero,Località,Prov.,Premio
1,T,173756,Somaglia,LO,5000000
2,T,378442,Pesaro,PU,2500000
3,G,330068,Palermo,PA,2000000
4,G,173817,Torino,TO,1500000
5,S,185025,Dolo,VE,1000000


In [76]:
cat1.insert(0, 'Categoria', 1)

In [77]:
cat1

,Categoria,Serie,Numero,Località,Prov.,Premio
1,1,T,173756,Somaglia,LO,5000000
2,1,T,378442,Pesaro,PU,2500000
3,1,G,330068,Palermo,PA,2000000
4,1,G,173817,Torino,TO,1500000
5,1,S,185025,Dolo,VE,1000000


---

# category 2

## remove all NaN columns

In [78]:
for ndx, df in enumerate(dfs[1:], 1):
    drop = []
    for column in df.columns:
        if df[column].isna().all():
            # print(ndx, column)
            drop.append(column)
    if drop:
        # print(ndx, drop)
        dfs[ndx] = df.drop(drop, axis=1)

In [79]:
dfs[1].head()

,1,2,3,4
0,SECONDA CATEGORIA,SECONDA CATEGORIA,SECONDA CATEGORIA,SECONDA CATEGORIA
1,Serie,Biglietto,Venduto a,Premio
2,S,030585,DISTRIBUTORE LOCALE MONTENERO DI BISACCIA (CB),100.000
3,C,439458,AVELLINO (AV),100.000
4,M,217442,MASSAFRA (TA),100.000


In [80]:
def parse_tables(dfs:list[pd.DataFrame]) -> pd.DataFrame:
    global col_names
    category = 1
    rows = []
    count = 0
    for ndx, df in enumerate(dfs[1:], 1):
        # print(df.head(1))
        for row in df.values:
            firstcol:str = row[0]
            if len(firstcol) > 2:
                #
                #   category or header
                #
                if 'categoria' in firstcol.lower():
                    # print(f'CATEGORY: {row}')
                    category += 1
                elif 'serie' in firstcol.lower():
                    # print(f'HEADER: {row}')
                    pass
                else:
                    raise ValueError(f'unknown pattern {row}')
            else:
                #
                #   row
                #
                row = np.insert(row, 0, category)
                rows.append(row)
                count += 1
    return rows

In [81]:
rows = parse_tables(dfs)
rows

[array([2, 'S', '030585', 'DISTRIBUTORE LOCALE MONTENERO DI BISACCIA (CB)',
        '100.000'], dtype=object),
 array([2, 'C', '439458', 'AVELLINO (AV)', '100.000'], dtype=object),
 array([2, 'M', '217442', 'MASSAFRA (TA)', '100.000'], dtype=object),
 array([2, 'D', '138065', 'FOLIGNO (PG)', '100.000'], dtype=object),
 array([2, 'M', '442388', 'BELLUNO (BL)', '100.000'], dtype=object),
 array([2, 'Q', '330398', 'FOGGIA (FG)', '100.000'], dtype=object),
 array([2, 'L', '320913', 'ISPICA (RG)', '100.000'], dtype=object),
 array([2, 'D', '052288', 'DISTRIBUTORE LOCALE GREVE IN CHIANTI (FI)',
        '100.000'], dtype=object),
 array([2, 'I', '265285', 'ROMA (RM)', '100.000'], dtype=object),
 array([2, 'B', '431066', 'SPINEA (VE)', '100.000'], dtype=object),
 array([2, 'B', '475642', 'OMEGNA (VB)', '100.000'], dtype=object),
 array([2, 'I', '479949', 'MONTEPULCIANO (SI)', '100.000'], dtype=object),
 array([2, 'I', '336915', 'IVREA (TO)', '100.000'], dtype=object),
 array([2, 'E', '322777',

In [82]:
# col_names = np.insert(col_names, 0, 'Categoria')
print(col_names)
newdf = pd.DataFrame(rows, columns=['Categoria', 'Serie', 'Numero', 'Località', 'Premio'])
newdf

0       Serie
1      Numero
2    Località
3       Prov.
4      Premio
Name: 0, dtype: object


,Categoria,Serie,Numero,Località,Premio
0,2,S,030585,DISTRIBUTORE LOCALE MONTENERO DI BISACCIA (CB),100.000
1,2,C,439458,AVELLINO (AV),100.000
2,2,M,217442,MASSAFRA (TA),100.000
3,2,D,138065,FOLIGNO (PG),100.000
4,2,M,442388,BELLUNO (BL),100.000
...,...,...,...,...,...
270,4,N,342401,TORINO (TO),20.0
271,4,M,41952,DISTRIBUTORE LOCALE ROMA (RM),20.0
272,4,T,404382,ROMA (RM),20.0
273,4,Q,270923,BAGHERIA (PA),20.0


In [83]:
newdf.insert(4, 'Prov.', newdf['Località'].str.extract('\((.*)\)'))
newdf.dtypes

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/var/folders/ch/0323xqnn3vbc6t95tfvmh0w40000gn/T/ipykernel_40119/2945868703.py:1: SyntaxWarning: invalid escape sequence '\('
  newdf.insert(4, 'Prov.', newdf['Località'].str.extract('\((.*)\)'))


Categoria     int64
Serie        object
Numero       object
Località     object
Prov.        object
Premio       object
dtype: object

In [84]:
'abcd (df)'.split('(')
'abcd df'.split('(')[0]
newdf['Località'] = newdf['Località'].apply(lambda x: x.split('(')[0])

In [85]:
newdf[newdf['Premio'].isna()]

,Categoria,Serie,Numero,Località,Prov.,Premio


In [86]:
newdf['Premio'] = newdf['Premio'].astype('str')

In [87]:
newdf['Premio'] = newdf.Premio.apply(lambda x: x + '00' if x.endswith('.0') else x)


In [88]:
newdf

,Categoria,Serie,Numero,Località,Prov.,Premio
0,2,S,030585,DISTRIBUTORE LOCALE MONTENERO DI BISACCIA,CB,100.000
1,2,C,439458,AVELLINO,AV,100.000
2,2,M,217442,MASSAFRA,TA,100.000
3,2,D,138065,FOLIGNO,PG,100.000
4,2,M,442388,BELLUNO,BL,100.000
...,...,...,...,...,...,...
270,4,N,342401,TORINO,TO,20.000
271,4,M,41952,DISTRIBUTORE LOCALE ROMA,RM,20.000
272,4,T,404382,ROMA,RM,20.000
273,4,Q,270923,BAGHERIA,PA,20.000


In [89]:
newdf.Premio = newdf.Premio.str.replace('.', '').astype('int')

In [90]:
newdf.dtypes

Categoria     int64
Serie        object
Numero       object
Località     object
Prov.        object
Premio        int64
dtype: object

In [91]:
cat1.head()

,Categoria,Serie,Numero,Località,Prov.,Premio
1,1,T,173756,Somaglia,LO,5000000
2,1,T,378442,Pesaro,PU,2500000
3,1,G,330068,Palermo,PA,2000000
4,1,G,173817,Torino,TO,1500000
5,1,S,185025,Dolo,VE,1000000


In [92]:
newdf.head()

,Categoria,Serie,Numero,Località,Prov.,Premio
0,2,S,030585,DISTRIBUTORE LOCALE MONTENERO DI BISACCIA,CB,100000
1,2,C,439458,AVELLINO,AV,100000
2,2,M,217442,MASSAFRA,TA,100000
3,2,D,138065,FOLIGNO,PG,100000
4,2,M,442388,BELLUNO,BL,100000


In [93]:
winners = pd.concat([cat1, newdf]).reset_index(drop=True)

In [94]:
winners = winners.reset_index(drop=True)

In [95]:
winners.head()

,Categoria,Serie,Numero,Località,Prov.,Premio
0,1,T,173756,Somaglia,LO,5000000
1,1,T,378442,Pesaro,PU,2500000
2,1,G,330068,Palermo,PA,2000000
3,1,G,173817,Torino,TO,1500000
4,1,S,185025,Dolo,VE,1000000


In [96]:
winners = winners.rename(columns={
    'Prov.':'Prov',
    'Località': 'Luogo'
})

In [97]:
winners[winners['Prov'] == 'TO']

,Categoria,Serie,Numero,Luogo,Prov,Premio
3,1,G,173817,Torino,TO,1500000
17,2,I,336915,IVREA,TO,100000
44,3,M,114512,SETTIMO TORINESE,TO,50000
99,4,T,478774,TORINO,TO,20000
100,4,M,355049,TORINO,TO,20000
129,4,O,460171,CIRIÈ,TO,20000
221,4,G,151078,RONDISSONE,TO,20000
273,4,G,394402,BUSSOLENO,TO,20000
275,4,N,342401,TORINO,TO,20000


In [98]:
winners.describe()

,Categoria,Premio
count,280.000000,2.800000e+02
mean,3.589286,7.500000e+04
std,0.727760,3.660307e+05
min,1.000000,2.000000e+04
25%,3.000000,2.000000e+04
50%,4.000000,2.000000e+04
75%,4.000000,5.000000e+04
max,4.000000,5.000000e+06


In [99]:
winners.to_csv('../out/winners.csv', index=False)